In [23]:
using Pkg
# Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames, TimerOutputs
using ProgressLogging
using StaticArrays

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [24]:
# Choose f from: Rastringin, alpine1, Ackley
f = Ackley

reset_timer!(Globtim._TO)

# Choose n from: 2, 3, 4
n = 2

# Constants and Parameters
const a, b = 1, 1
const scale_factor = a / b
rand_center = zeros(Float64, n);
d = 10 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor 
                )

test_input(2, [0.0, 0.0], 40, (0.1, 0.5), 0.002, (0.0, 0.0), 1.0, 1.0, 6, Globtim.Ackley)

Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [25]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.1700717268466939


In [26]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

17-element Vector{Vector{Float64}}:
 [0.9640968143739062, 0.9640968143739048]
 [6.330804320179865e-18, 0.9629131980007132]
 [-0.9640968143739085, 0.9640968143739027]
 [-0.6981226790435401, 0.6981226790435365]
 [0.9629131980007138, 1.5007229786735268e-16]
 [2.0689020949081755e-17, 2.7491256920457325e-16]
 [-0.962913198000715, 2.1092288166840944e-16]
 [-0.6528936546007025, 3.207128761745569e-16]
 [0.6528936546007024, 3.3889800168938247e-16]
 [0.6981226790435193, -0.6981226790435237]
 [0.9640968143739043, -0.964096814373908]
 [3.209677859062992e-17, -0.9629131980007174]
 [-0.9640968143739034, -0.9640968143739083]
 [3.422725992320801e-17, -0.6528936546006987]
 [-0.6981226790435179, -0.6981226790435316]
 [2.7162092840535873e-17, 0.6528936546007063]
 [0.6981226790435402, 0.6981226790435283]

In [27]:
df_cheb = process_crit_pts(real_pts_cheb, f, TR)

Row,x1,x2,z
,Float64,Float64,Float64
1,0.964097,0.964097,3.57539
2,6.3308e-18,0.962913,2.58271
3,-0.964097,0.964097,3.57539
4,-0.698123,0.698123,4.59859
5,0.962913,1.50072e-16,2.58271
6,2.0689e-17,2.74913e-16,4.44089e-16
7,-0.962913,2.10923e-16,2.58271
8,-0.652894,3.20713e-16,3.24424
9,0.652894,3.38898e-16,3.24424


In [28]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.5);

Processing point 1 of 17
Optimization has converged within bounds: ✓
Processing point 2 of 17
Optimization has converged within bounds: ✓
Processing point 3 of 17
Optimization has converged within bounds: ✓
Processing point 4 of 17
Optimization has converged within bounds: ✓
Processing point 5 of 17
Optimization has converged within bounds: ✓
Processing point 6 of 17
Optimization has converged within bounds: ✓
Processing point 7 of 17
Optimization has converged within bounds: ✓
Processing point 8 of 17
Optimization has converged within bounds: ✓
Processing point 9 of 17
Optimization has converged within bounds: ✓
Processing point 10 of 17
Optimization has converged within bounds: ✓
Processing point 11 of 17
Optimization has converged within bounds: ✓
Processing point 12 of 17
Optimization has converged within bounds: ✓
Processing point 13 of 17
Optimization has converged within bounds: ✓
Processing point 14 of 17
Optimization has converged within bounds: ✓
Processing point 15 of 17
Opt

In [29]:
df_cheb

Row,x1,x2,z,y1,y2,close,steps,converged
,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Bool
1,0.964097,0.964097,3.57539,0.968478,0.968478,true,3.0,true
2,6.3308e-18,0.962913,2.58271,6.3308e-18,0.952167,true,3.0,true
3,-0.964097,0.964097,3.57539,-0.968478,0.968478,true,3.0,true
4,-0.698123,0.698123,4.59859,-0.968478,0.968478,true,4.0,true
5,0.962913,1.50072e-16,2.58271,0.952167,1.50072e-16,true,3.0,true
6,2.0689e-17,2.74913e-16,4.44089e-16,2.0689e-17,2.74913e-16,true,0.0,true
7,-0.962913,2.10923e-16,2.58271,-0.952167,2.10923e-16,true,3.0,true
8,-0.652894,3.20713e-16,3.24424,-0.952167,3.20713e-16,true,5.0,true
9,0.652894,3.38898e-16,3.24424,0.952167,3.38898e-16,true,5.0,true


In [30]:
Globtim._TO

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            5.83s /   2.2%           59.1MiB /  87.2%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
Constructor                1   86.2ms   66.7%  86.2ms   46.6MiB   90.4%  46.6MiB
  MainGenerate             1   86.0ms   66.5%  86.0ms   46.6MiB   90.4%  46.6MiB
    lambda_vanderm...      1   4.08ms    3.2%  4.08ms    918KiB    1.7%   918KiB
    generate_grid_...      1   3.62ms    2.8%  3.62ms    868KiB    1.6%   868KiB
    linear_solve_v...      1    139μs    0.1%   139μs   66.7KiB    0.1%  66.7KiB
    evaluation             1   76.0μs    0.1%  76.0μs   13.7KiB    0.0%  13.7KiB
solve_polynomial_s...      

## Timings for different $n$

In [32]:
for n in [2,3]
    # Choose f from: Rastringin, alpine1, Ackley
    f = Ackley

    reset_timer!(Globtim._TO)

    # Constants and Parameters
    const a, b = 1, 1
    const scale_factor = a / b
    rand_center = zeros(Float64, n);
    d = 10 # initial degree 
    SMPL = 40 # Number of samples
    TR = test_input(f, 
                    dim = n,
                    center=rand_center,
                    GN=SMPL, 
                    sample_range=scale_factor 
                    )

    pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
    @polyvar(x[1:n]); # Define polynomial ring 
    @polyvar(x[1:n]) # Define polynomial ring 
    real_pts_cheb = solve_polynomial_system(
        x, n, d, pol_cheb.coeffs;
        basis=pol_cheb.basis,
        precision=pol_cheb.precision,
        normalized=pol_cheb.normalized,
    )
    
    df_cheb = process_crit_pts(real_pts_cheb, f, TR)

    df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.5);

    println("n = $n")
    println(Globtim._TO)
end

current L2-norm: 0.1700717268466939
Processing point 1 of 17
Optimization has converged within bounds: ✓
Processing point 2 of 17
Optimization has converged within bounds: ✓
Processing point 3 of 17
Optimization has converged within bounds: ✓
Processing point 4 of 17
Optimization has converged within bounds: ✓
Processing point 5 of 17
Optimization has converged within bounds: ✓
Processing point 6 of 17
Optimization has converged within bounds: ✓
Processing point 7 of 17
Optimization has converged within bounds: ✓
Processing point 8 of 17
Optimization has converged within bounds: ✓
Processing point 9 of 17
Optimization has converged within bounds: ✓
Processing point 10 of 17
Optimization has converged within bounds: ✓
Processing point 11 of 17
Optimization has converged within bounds: ✓
Processing point 12 of 17
Optimization has converged within bounds: ✓
Processing point 13 of 17
Optimization has converged within bounds: ✓
Processing point 14 of 17
Optimization has converged within bou

InterruptException: InterruptException: